In [1]:
class CDP(object):

    MIN_RATIO = 3.0/2.0

    def __init__(self, eth_on_hand=100, price=150):
        self.i = 0
        self.usd_on_hand = 0
        self.usd_generated = 1
        self.eth_deposited = 0.01

        self.start_eth_on_hand = eth_on_hand - self.eth_deposited
        self.eth_on_hand = eth_on_hand - self.eth_deposited
        self.start_price = price
        self.price = price

        self._update_calculations()

    def _update_calculations(self):
        self.liquidation_price = self.usd_generated * self.MIN_RATIO / self.eth_deposited
        self.usd_value = self.price * self.eth_deposited
        self.leverage_ratio = self.usd_value / self.usd_generated
        self.usd_available_to_generate = self.usd_value / self.MIN_RATIO - self.usd_generated

    # TODO: simplify this function
    def _print_action(self, action, eth_usd, amt, max_amt, price=None):
        if amt > max_amt:
            return 'Invalid request. Unable to {} {}.'.format(action, eth_usd)

        if action == 'deposit':
            self.eth_deposited += amt
            self.eth_on_hand -= amt
        elif action == 'generate':
            self.usd_generated += amt
            self.usd_on_hand += amt
        else:  #  action == 'trade'
            self.eth_on_hand += amt/price
            self.usd_on_hand -= amt

        self.i += 1
        print('{}. {} {} {}'.format(self.i, action, amt, eth_usd))
        self._update_calculations()

    def deposit_eth(self, eth):
        self._print_action(action='deposit', eth_usd='ETH', amt=eth, max_amt=self.eth_on_hand)
    
    def generate_usd(self, usd):
        self._print_action(action='generate', eth_usd='USD', amt=usd, max_amt=self.usd_available_to_generate)
        
    def trade_usd_for_eth(self, usd, price):
        self._print_action(action='trade', eth_usd='USD', amt=usd, max_amt=self.usd_on_hand, price=price)
        
    def update_eth_price(self, price):
        print('Update ETH price:  ${} => ${}\n'.format(self.price, price))
        self.price = price

    def summarize(self):
        print('---------------------------')
        print('Current ETH price: ${0:.2f}'.format(self.price))
        print('Liquidation price: ${0:.2f}'.format(self.liquidation_price))

        print('\nUSD value: ${0:.2f}'.format(self.usd_value))
        print('ETH locked: {0:.2f} ETH'.format(self.eth_deposited))
        print('ETH on hand: {0:.2f} ETH'.format(self.eth_on_hand))
        print('USD on hand: ${0:.2f}'.format(self.usd_on_hand))

        print('\nUSD generated: ${0:.2f}'.format(self.usd_generated))
        print('USD available to generate: ${0:.2f}'.format(self.usd_available_to_generate))
        print('---------------------------\n')

    def close(self):
        eth_owed = self.usd_generated / self.price
        end_eth = self.eth_deposited - eth_owed
        pct_change_eth_price = (self.price - self.start_price) / self.start_price
        pct_change_eth_balance = (end_eth - self.start_eth_balance) / self.start_eth_balance
        print('Starting balance: {0:.2f} ETH'.format(self.start_eth_balance))
        print('Ending balance: {0:.2f} ETH'.format(end_eth))
        print('% change in ETH price {0:.2%}'.format(pct_change_eth_price))
        print('% change in ETH holdings: {0:.2%}'.format(pct_change_eth_balance))

In [2]:
eth_start_price = 173.28
cdp = CDP(eth_on_hand=115, price=eth_start_price)

# Deposit 109.99 of starting ETH (keeping 5 ETH for future tx costs)
cdp.deposit_eth(109.99)

# Withdraw 80% of USD available
withdrawal_1 = cdp.usd_available_to_generate * 0.8
cdp.generate_usd(usd=withdrawal_1)

# Trade USD to ETH on eth2dai.com @ $178.2 per ETH
eth_trade_price = 178.2
cdp.trade_usd_for_eth(usd=withdrawal_1, price=eth_trade_price)

# Deposit ETH based starting eth_price (which equals $10,164)
cdp.deposit_eth(withdrawal_1 / eth_start_price)
cdp.summarize()

1. deposit 109.99 ETH
2. generate 10164.96 USD
3. trade 10164.96 USD
4. deposit 58.66204986149584 ETH
---------------------------
Current ETH price: $173.28
Liquidation price: $90.41

USD value: $29225.76
ETH locked: 168.66 ETH
ETH on hand: 3.38 ETH
USD on hand: $0.00

USD generated: $10165.96
USD available to generate: $9317.88
---------------------------



In [3]:
# Withdraw 60% of available USD
withdrawal_2 = cdp.usd_available_to_generate * 0.6
cdp.generate_usd(usd=withdrawal_2)

# Trade USD to ETH
eth_trade_price = 180.28
cdp.trade_usd_for_eth(usd=withdrawal_2, price=eth_trade_price)

# Deposit ETH purchased above
cdp.deposit_eth(withdrawal_2 / eth_trade_price)
cdp.update_eth_price(173.895)
cdp.summarize()

5. generate 5590.728 USD
6. trade 5590.728 USD
7. deposit 31.011360106500998 ETH
Update ETH price:  $173.28 => $173.895

---------------------------
Current ETH price: $173.90
Liquidation price: $118.37

USD value: $34599.41
ETH locked: 199.67 ETH
ETH on hand: 3.38 ETH
USD on hand: $0.00

USD generated: $15756.69
USD available to generate: $7309.58
---------------------------



![title](images/2019.04.23.png)

In [ ]:
s